# Revised Pay As You Earn (REPAYE) student loan calculator

In this notebook, I will implement my own version of a student loan calculator, like this [exampe](https://studentloanhero.com/calculators/student-loan-revised-pay-as-you-earn-calculator/) from Studen Loan Hero. My fiance went to medical school at USC and took out full loans to cover his tuition and living expenses for four years. He has been on an income-based repayment plan during his two years in residency. We recently learned that it's very disadvantageous for us to be legally married, as my income is automatically combined with his and his monthly loan payments would greatly increase. I wanted to run the numbers myself, in an interactive and visual way. This is what I came up with!

**Disclaimer**: This should in no way be taken as financial advice or be taken to be completely accurate. We still plan on consulting with a tax professional on all of this, but I just wanted to code this up myself for my own peace of mind and to get a rough idea of the numbers.

Let's load in all the necessary modules.

In [1]:
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource, output_file
from bokeh.models import Legend, LegendItem
from bokeh.models.tools import HoverTool, BoxSelectTool, BoxZoomTool, PanTool, \
WheelZoomTool, SaveTool, ResetTool
from bokeh.models.tickers import FixedTicker
from bokeh.models.widgets import CheckboxGroup, RangeSlider, Tabs, TextInput, \
Button, RadioGroup, RadioButtonGroup, Select, Paragraph, Div, DataTable, \
TableColumn, NumberFormatter
from bokeh.layouts import column, row, WidgetBox, Spacer, gridplot
from bokeh.models import Panel, Range1d, LinearAxis
from bokeh.io import show, curdoc, output_notebook
import os
import pandas as pd
import numpy as np
from math import pi
from copy import deepcopy

In [2]:
output_notebook()

Loading BokehJS ...

On the REPAYE plan, you pay 10% of your discretionary income every month towards your loans. Discretionary income is based on the poverty line for your family size and region (contiguous 48 states, Alaska, Hawaii). Specifically, it's your adjusted gross income (AGI) minus 150% above the poverty line. For example, for a family of 1 living in California, the poverty line is \$12,490. You would subtract (12490 * 1.5) = $18,735 from your AGI to get your discretionary income.

***It's important to note*** that I did not adjust the poverty line to increase with inflation, as other calculators do using information from the Congressional Budget Office. In later years, the discretionary income will be higher (than if it were inflation adjusted), resulting in higher monthly payments and a lower total for forgiven loan.

Let's read in the poverty level information and define a function to calculate discretionary income.

In [3]:
# we'll make this global so it's easier to read inside other functions
global poverty_df
poverty_df = pd.read_table('poverty_levels.txt')

In [4]:
def discretionary_income(gross_income, size, region, poverty_df):

	if size > 8:
		if region == 'Alaska':
			bonus = 5530 * (size - 8)
		elif region == 'Hawaii':
			bonus = 5080 * (size - 8)
		else:
			bonus = 4420 * (size - 8)
	else:
		bonus = 0

	# 150% above poverty level
	poverty_level = poverty_df.amount[
	(poverty_df['size'] == size) &
	(poverty_df['region'] == region)].values[0] + bonus

	discretionary = gross_income - (1.5 * poverty_level)

	return discretionary

Federal student loans do not compound interest. Rather, the interest is based on the outstanding principal. If a medical school loan had compounding interest, that would be even more terrible.

In [5]:
def calculate_interest(loan, interest_rate, loan_years, subsidy=0):
	# interest rate should be in decimal, not percentage
	interest_annual = loan * interest_rate
	interest = loan_years * interest_annual
	interest_paid = interest - (interest * subsidy)
	return interest_paid

First, there are three payment types: REPAYE, PAYE and standard

### Standard loan payment ###
- This is the default loan payment method for federal student loans. It sets the monthly payments such that you pay off the loan in ten years. 

### Pay As You Earn (PAYE)

This is the original of the REPAYE plan. There are a few major differences between PAYE and REPAYE: 
- Under PAYE, your joint income is only used for calculating monthly payments if you file taxes jointly. Under REPAYE, this loophole was closed and joint income for a married couple is used regardless of tax filing status.
- PAYE does not have an interest subsidy like REPAYE.

### Revised Pay As You Earn (REPAYE)

- Joint income is used for monthly payments, regardless of tax filing status
- However, there is a nice interest subsidy. The aim is to control the accruing interest from getting out of hand. If your monthly income payments do not cover the interest, the government will give you 50% of the remaining unpaid interest. The other 50% of unpaid interest **is not accrued**. This results in your overall loan balance remaining static while your monthly payments cannot cover the interest. Your payments aren't going towards the principal, but at least your overall loan amount does not increase. This is really helpful during residency is your loans are particularly massive.

Finally, how much your income grows over time is important because as your income rises, so do your monthly payments.

After 20 (PAYE) or 25 (REPAYE) years of on-time payments, the remaining loan balance is forgiven. *However*, this forgiven loan is considered taxable income, so you must save up for this tax bomb. 

Marital status, spouse's income (if any), and plan type are the main factors that determine your income-based plan. This is the main function that updates whenever there are new inputs. It takes into consideration the marital status, combined income, and plan type to calculate the appropriate payment plan. 

In [6]:
def calculate_loan_payment(plan_type, loan_years, loan, interest_rate, 
	income_ag, income_growth, size, region, marital_status, income_spouse):

	# determine income. If REPAYE, joint income considered regardless of 
	# filing status
	if marital_status != 'single':
		if marital_status == 'married filing separate':
			if plan_type != 'REPAYE':
				income_total_gross = income_ag
			else:
				income_total_gross = income_ag + income_spouse
		else: # married filing jointly
			income_total_gross = income_ag + income_spouse
	else:
		income_total_gross = income_ag

	if plan_type == 'standard':
		plan = standard_plan(loan, income_ag, income_growth, interest_rate)

	else:
		income_discretionary = discretionary_income(
			income_total_gross, 
			size, 
			region, 
			poverty_df)

		plan = income_based_payments(
			income_discretionary,
			income_growth,
			loan_years,
			loan,
			interest_rate,
			plan_type)

	return plan

Let's define the function for PAYE/REPAYE.

In [7]:
def income_based_payments(income, income_growth, loan_years,
loan, interest_rate, plan_type, income_percentage=0.10):
	# income percentage, income_growth and interest_rate should be in decimal, 
	# not percentage
	payment_total = 0
	income_growing = income
	loan_remaining = loan

	# REPAYE has interest subsidy, PAYE does not
	if plan_type == 'REPAYE':
		subsidy = 0.50
	else:
		subsidy = 0

	# save yearly payment info to display with Bokeh data table
	payment_month_list = []
	interest_subsidy_list = []
	payment_total_list = []
	loan_remaining_list = []
	interest_list = []
	income_list = []

	for i in range(loan_years):

		# print "Year:", i
		payment_annual = income_growing * income_percentage
		income_list.append(income_growing)
		# print "Payment annual:", payment_annual
		payment_month = payment_annual / 12.
		# print "Monthly payment:", payment_month
	
		
		if i == 0:
			payment_first_month = payment_annual / 12.

		interest_annual = min(loan, loan_remaining) * interest_rate
		interest_remaining = interest_annual - payment_annual
		

		if interest_remaining > 0 and subsidy != 0:
			# add interest subsidy, 50% from government
			interest_subsidy = interest_remaining * subsidy
			# do not add remaining unpaid interest, only add interest equal to 
			# annual payment and subsidy, so loan balance does not change
			payment_annual += interest_subsidy
			interest_list.append(payment_annual)
		else:
			interest_subsidy = 0
			loan_remaining += interest_annual
			loan_remaining -= payment_annual
			interest_list.append(interest_annual)

		# print "Interest subsidy:", interest_subsidy


		payment_total += payment_annual
		# print "Total payment:", payment_total
		
		# print "Remaining loan:", loan_remaining

		income_growing += income_growing * income_growth

		payment_month_list.append(round(payment_month, 2))
		interest_subsidy_list.append(max(0.0, round(interest_subsidy, 2)))
		payment_total_list.append(round(payment_total, 2))
		loan_remaining_list.append(round(loan_remaining, 2))


	payments = dict(
		years = [i+1 for i in range(loan_years)],
		payment_month = payment_month_list,
		interest_subsidy = interest_subsidy_list,
		payment_total = payment_total_list,
		loan_remaining = loan_remaining_list,
		interest_annual = interest_list,
		income_annual = income_list
	)

	# add field for annual payment for debugging
	payments['payment_annual'] = [x * 12 for x in payments['payment_month']]


	return payments

Function for standard loan repayment (in progress)

In [8]:
def standard_plan(loan, income, income_growth, interest_rate):

	payment_annual = loan / 10.
	payment_month = payment_annual / 12.

	payment_toal_list = []
	loan_remaining_list = []
	interest_list = []
	income_list = []

	income_growing = income
	loan_remaining = loan

	for i in range(10):
		# print "Year:", i
		income_list.append(income_growing)
		

		interest_annual = min(loan, loan_remaining) * interest_rate

		loan_remaining += interest_annual
		loan_remaining -= payment_annual
		
		interest_list.append(interest_annual)

		payment_total += payment_annual
		# print "Total payment:", payment_total
		
		# print "Remaining loan:", loan_remaining

		income_growing += income_growing * income_growth

		payment_total_list.append(round(payment_total, 2))
		loan_remaining_list.append(round(loan_remaining, 2))


	payments = dict(
		years = [i+1 for i in range(10)],
		payment_month = 12 * [payment_month],
		interest_subsidy = 12 * [0.0],
		payment_total = payment_total_list,
		loan_remaining = loan_remaining_list,
		interest_annual = interest_list,
		income_annual = income_list
	)

	return payments

Let's define widget boxes for all the relevant variables. The initial values are set to my own scenario for my own convenience. I set the loan term to 20 years because that's how long is left after his four years of residency + one year of fellowship.

We'll define everything we need, and then display it inline.

In [9]:
def modify_doc(doc):
    
    # update based on input
    def recalculate(attr, old, new):
        # get current list of relevant variables
        plan_type = plan_choice.labels[plan_choice.active]
        loan_years = int(term_input.value)
        loan = float(loan_input.value)
        interest_rate = float(interest_input.value) / 100.
        income = float(income_input.value)
        income_growth = float(growth_input.value) / 100.
        size = int(size_input.value)
        region = region_choice.value
        marital_status = marital_choice.value
        income_spouse = float(income_spouse_input.value)

        income_discretionary_new = discretionary_income(
        income, 
        size, 
        region, 
        poverty_df)

        new_plan = calculate_loan_payment(plan_type, loan_years, loan, interest_rate, 
            income, income_growth, size, region, marital_status, income_spouse)

        source.data.update(new_plan)
        
    # plan payment type
    plan_choice = RadioGroup(
        labels=['REPAYE', 'PAYE', 'standard (10 loan years)'], active=0)
    plan_choice.on_change('active', recalculate)

    # how long is the loan
    term_input = TextInput(value='20', title='Loan term (loan years)')
    term_input.on_change('value', recalculate)

    loan_input = TextInput(value='411000', title='Loan amount')
    loan_input.on_change('value', recalculate)

    # interest rate on loan
    interest_input = TextInput(value='6.04', title='Interest rate (%)')
    interest_input.on_change('value', recalculate)

    # adjusted gross income
    income_text = Div(text='''Adjusted gross income. Discretionary income will 
        be calculated based on state and family size''')
    income_input = TextInput(value='225000', title='Adjusted gross income')
    income_input.on_change('value', recalculate)

    # how much does income grow
    growth_text = Div(text='''Your income grows over time, so your income based 
        repayments will increase as well. Need to estimate how your income will 
        grow over time.''')
    growth_input = TextInput(value='3.5', title='''Income growth (3.5% is 
        historic average)''')
    growth_input.on_change('value', recalculate)

    # family size
    size_text = Div(text='''Income percentage is based on discretionary income, 
        which is your AGI minus 150% of the poverty line for your family size.
        ***Currently, poverty line does not increase with inflation so discretionary income
        over time is higher than it should be. This results in higher monthly payments compared
        to online REPAYE calculators.***''')
    size_input = TextInput(value='1', title='Family size')
    size_input.on_change('value', recalculate)

    # where do you live
    region_choice = Select(title='State:', value='contiguous 48', 
        options=['contiguous 48', 'Alaska', 'Hawaii'])
    region_choice.on_change('value', recalculate)

    # marital status
    marital_text = Div(text='''REPAYE considers joint income whether filing 
        separately or jointly. Compares tax differences when filing 
        jointly/separately''')
    marital_choice = Select(title='Marital status', value='single',
        options=['single', 'married filing separate', 'married filing jointly'])
    marital_choice.on_change('value', recalculate)

    # how much does your spouse make
    income_spouse_input = TextInput(value='0', title='Spouse income')
    income_spouse_input.on_change('value', recalculate)

    box1 = WidgetBox(plan_choice, loan_input, term_input, interest_input,
                    income_text, income_input,
                     growth_text, growth_input,)
    
    box2 = WidgetBox(
        size_text, size_input,
        region_choice,
        marital_text, marital_choice, income_spouse_input)

    
    # grab initial input
    plan_type = plan_choice.labels[plan_choice.active]
    loan_years = int(term_input.value)
    loan = float(loan_input.value)
    interest_rate = float(interest_input.value) / 100.
    income = float(income_input.value)
    income_growth = float(growth_input.value) / 100.
    size = int(size_input.value)
    region = region_choice.value
    marital_status = marital_choice.value
    income_spouse = float(income_spouse_input.value)

    initial_plan = calculate_loan_payment(plan_type, loan_years, loan, interest_rate, income,
        income_growth, size, region, marital_status, income_spouse)
    
    # display repayment info
    # display annual loan payment info
    source = ColumnDataSource(initial_plan)
    columns = [
        TableColumn(field='years', title='Year'),
        TableColumn(field='income_annual', title='Annual discretionary income',
            formatter=NumberFormatter()),
        TableColumn(field='payment_month', title='Monthly payment',
            formatter=NumberFormatter()),
        TableColumn(field='payment_annual', title='Annual payment',
            formatter=NumberFormatter()),
        TableColumn(field='interest_annual', title='Annual interest accrued',
            formatter=NumberFormatter()),
        TableColumn(field='interest_subsidy', title='Annual interest subsidy'),
        TableColumn(field='payment_total', title='Total loan payment', 
            formatter=NumberFormatter()),
        TableColumn(field='loan_remaining', title='Remaining loan balance',
            formatter=NumberFormatter())
    ]

    payment_table = DataTable(source=source, columns=columns, width=1200, height=800)

    # create layout for everything
    layout = row(box1, box2, payment_table)
    
    # add layout to current document
    doc.add_root(layout)

In [10]:
show(modify_doc, notebook_url='http://localhost:8888')

### Interest subsidy is great ### 

From the table above, you can see that the interest subsidy really pays off for us. My partner does not make enough in annual payments to cover the interest for *six years*. Although he's making payments that don't touch the principal, it's really nice that interest doesn't keep adding up thanks to the subsidy. You can also see how over the first six years the interest subsidy keeps decreasing until he makes enough money to cover the interest payments.

### Monthly payments ###

Although the monthly payments nearly double over 20 years, it's still less than the monthly payment under the standard plan (code still in progress, but its ~$4,700/month).

### In progress ###

I plan to add functionality that will calculate how much more you pay with REPAYE vs. standard loan, and which one makes better financial sense, including the tax bomb. I've run the numbers by hand, including the tax bomb, and you pay ~\$48,000 more under the REPAYE plan compared to standard (\$548K vs \$590K), although it's spread out over ten more years. The tax bomb for REPAYE on \$257K forgiven is ~\$130K, based on his income growth after 20 years and the California Income tax calculator (with and without the tax bomb). One thing to consider is that the burden of this bill will be eased with inflation. 

### Final verdict ###

I was pleasantly surprised to see that the difference in the total amount paid on the loan between REPAYE vs. standard wasn't too big, roughly one extra year of paying on the standard plan. While it's a lot to save up for the tax bomb, the saving is spread out over 20 years and the impact is reduced with inflation (i.e. it's easier to pay that off with future dollars). Finally, although we aren't planning on this, the tax bomb could be changed with legislation. Many students signed on to these types of plans within the past few years, so there is some years before millions of people will have to pay up for this loan forgiveness. 

We personally think that the tax bomb is worth it in order to enjoy our money while we're younger, for things like buying a house. The standard monthly payment is such a high percentage of his monthly income, almost 40%, and he could theoretically pay off the loan aggressively over the next five or so years. While the choice is very personal and depends on your own financial philosophy and circumstance, planning ahead for the loan forgiveness seems to be the smartest choice for us both financially and for our happiness!